In [12]:
import numpy as np
import pandas as pd
df = pd.read_csv("/content/netflix_titles.csv")
a = pd.DataFrame(df)
print(a)

     show_id     type                  title         director  \
0         s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1         s2  TV Show          Blood & Water              NaN   
2         s3  TV Show              Ganglands  Julien Leclercq   
3         s4  TV Show  Jailbirds New Orleans              NaN   
4         s5  TV Show           Kota Factory              NaN   
...      ...      ...                    ...              ...   
8802   s8803    Movie                 Zodiac    David Fincher   
8803   s8804  TV Show            Zombie Dumb              NaN   
8804   s8805    Movie             Zombieland  Ruben Fleischer   
8805   s8806    Movie                   Zoom     Peter Hewitt   
8806   s8807    Movie                 Zubaan      Mozez Singh   

                                                   cast        country  \
0                                                   NaN  United States   
1     Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa 

In [14]:
import pandas as pd
import numpy as np

src = '/content/netflix_titles.csv'
out_csv = '/content/netflix_titles_cleaned.csv'

df = pd.read_csv(src)

# 1. Standardize column names
df.columns = (df.columns
                .str.strip()
                .str.lower()
                .str.replace(' ', '_')
                .str.replace('[^0-9a-zA-Z_]', '', regex=True))

# 2. Quick inspect
print(df.shape)
print(df.dtypes)
print((df.isnull().mean()*100).sort_values(ascending=False).head(10))

# 3. Drop exact duplicates
df = df.drop_duplicates()

# 4. Drop columns with >60% missing (optional)
missing_pct = df.isnull().mean()*100
cols_to_drop = missing_pct[missing_pct > 60].index.tolist()
df = df.drop(columns=cols_to_drop)

# 5. Impute common text columns
for col in ['cast','director','country']:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

# 6. Fill rating with mode
if 'rating' in df.columns:
    mode_rating = df['rating'].mode()
    df['rating'] = df['rating'].fillna(mode_rating[0] if not mode_rating.empty else 'Not Rated')

# 7. Parse dates
if 'date_added' in df.columns:
    df['date_added_parsed'] = pd.to_datetime(df['date_added'], errors='coerce')
    df['added_year'] = df['date_added_parsed'].dt.year
    df['added_month'] = df['date_added_parsed'].dt.month
    df['date_added_ddmmyyyy'] = df['date_added_parsed'].dt.strftime('%d-%m-%Y')

# 8. Parse duration -> numeric + unit
if 'duration' in df.columns:
    df['duration_num'] = pd.to_numeric(df['duration'].str.extract(r'(\d+)', expand=False), errors='coerce')
    df['duration_unit'] = df['duration'].str.replace(r'(\d+)', '', regex=True).str.strip()
    df['duration_minutes'] = df['duration_num'].where(df['duration_unit'].str.lower().str.contains('min', na=False), np.nan)

# 9. Trim whitespace on text columns
obj_cols = df.select_dtypes(include='object').columns
for c in obj_cols:
    df[c] = df[c].astype(str).str.strip()

# 10. Handle simple numeric conversion
if 'release_year' in df.columns:
    df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce').astype('Int64')

# 11. Optional: outlier capping for duration_minutes (IQR)
if 'duration_minutes' in df.columns and df['duration_minutes'].notna().sum() > 0:
    q1 = df['duration_minutes'].quantile(0.25)
    q3 = df['duration_minutes'].quantile(0.75)
    iqr = q3 - q1
    low = q1 - 1.5*iqr
    high = q3 + 1.5*iqr
    df['duration_minutes_capped'] = df['duration_minutes'].clip(lower=low, upper=high)

# 12. Save cleaned CSV
df.to_csv(out_csv, index=False)
print("Saved cleaned CSV to", out_csv)

(8807, 12)
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object
director        29.908028
country          9.435676
cast             9.367549
date_added       0.113546
rating           0.045418
duration         0.034064
show_id          0.000000
type             0.000000
title            0.000000
release_year     0.000000
dtype: float64
Saved cleaned CSV to /content/netflix_titles_cleaned.csv
